In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import os
import pandas as pd
#Functional coding
import functools
from functools import partial

In [2]:
'a' in ['a','b','c']

True

In [3]:
Datapath="DATA/HVAC/HVAC_Data.txt"
Labelpath="DATA/HVAC/HVAC_Label.txt"
Rewardpath="DATA/HVAC/HVAC_Reward.txt"

In [4]:
#Given local path, find full path
def PathFinder(path):
    #python 2
    #script_dir = os.path.dirname('__file__')
    #fullpath = os.path.join(script_dir,path)
    #python 3
    fullpath=os.path.abspath(path)
    print(fullpath)
    return fullpath

#Read Data for Deep Learning
def ReadData(path):
    fullpath=PathFinder(path)
    return pd.read_csv(fullpath, sep=',', header=0)

In [5]:
S_A_pd = ReadData(Datapath)
SP_pd = ReadData(Labelpath)
R_pd = ReadData(Rewardpath)
S_A_matrix=S_A_pd.as_matrix()
SP_matrix=SP_pd.as_matrix()
R_matrix=R_pd.as_matrix()

/home/wuga/Documents/Notebook/VAE-PLANNING/DATA/HVAC/HVAC_Data.txt
/home/wuga/Documents/Notebook/VAE-PLANNING/DATA/HVAC/HVAC_Label.txt
/home/wuga/Documents/Notebook/VAE-PLANNING/DATA/HVAC/HVAC_Reward.txt


In [6]:
default_settings = {                
    "cap": tf.constant(80.0,dtype=tf.float32), 
    "outside_resist" : tf.constant(4.0,dtype=tf.float32),
    "hall_resist" : tf.constant(2.0,dtype=tf.float32),
    "wall_resist" : tf.constant(1.5,dtype=tf.float32),
    "cap_air" : tf.constant(1.006,dtype=tf.float32), 
    "cost_air" : tf.constant(1.0,dtype=tf.float32), 
    "time_delta" : tf.constant(1.0,dtype=tf.float32),
    "temp_air" : tf.constant(40.0,dtype=tf.float32),
    "temp_up" : tf.constant(23.5,dtype=tf.float32),
    "temp_low" : tf.constant(20.0,dtype=tf.float32),
    "temp_outside" : tf.constant(6.0,dtype=tf.float32),
    "temp_hall" : tf.constant(10.0,dtype=tf.float32),
    "penalty" : tf.constant(20000.0,dtype=tf.float32),
    "air_max" : tf.constant(10.0,dtype=tf.float32)
   }

In [7]:
class HVAC(object):
    def __init__(self, 
                 adj_outside, #Adjacent to outside 
                 adj_hall, #Adjacent to hall
                 adj, #Adjacent between rooms
                 rooms, #Room names
                 default_settings):
        self.__dict__.update(default_settings)
        self.adj_outside = adj_outside
        self.adj_hall = adj_hall
        self.adj = adj
        self.rooms = rooms
        self.zero = tf.constant(0,dtype=tf.float32)
        
    def ADJ(self, space1, space2):
        for pair in self.adj:
            if space1 in pair and space2 in pair:
                return True
        return False
                 
    def ADJ_OUTSIDE(self,  space):
        if space in self.adj_outside:
            return True
        else:
            return False
            
    def ADJ_HALL(self, space):
        if space in self.adj_hall:
            return True
        else:
            return False  
        
    def R_OUTSIDE(self, space):
        return self.outside_resist
    
    def R_HALL(self, space):
        return self.hall_resist
    
    def R_WALL(self, space1, space2):
        return self.wall_resist
        
    def CAP(self, space):
        return self.cap
    
    def CAP_AIR(self):
        return self.cap_air
    
    def COST_AIR(self):
        return self.cost_air
    
    def TIME_DELTA(self):
        return self.time_delta
    
    def TEMP_AIR(self):
        return self.temp_air
    
    def TEMP_UP(self, space):
        return self.temp_up
    
    def TEMP_LOW(self, space):
        return self.temp_low
    
    def TEMP_OUTSIDE(self, space):
        return self.temp_outside
    
    def TEMP_HALL(self, space):
        return self.temp_hall
    
    def PENALTY(self):
        return self.penalty
    
    def AIR_MAX(self, space):
        return self.air_max
    
    # Single state function, need map to matrix later
    def _transition(self, space, states, actions):
        
        previous_state = states[space]
        heating_info = self.TIME_DELTA()/self.CAP(space)*(actions[space]*self.CAP_AIR()*(self.TEMP_AIR()-previous_state))
        neighbor_info = self.zero
        for p in self.rooms:
            if self.ADJ(space,p):
                neighbor_info += (states[p]-previous_state)/self.R_WALL(space,p)
        outside_info = self.zero
        if self.ADJ_OUTSIDE(space):
            outside_info=(self.TEMP_OUTSIDE(space)-previous_state)/self.R_OUTSIDE(space)
        wall_info = self.zero
        if self.ADJ_HALL(space):
            wall_info=(self.TEMP_HALL(space)-previous_state)/self.R_HALL(space)
            
        new_state = previous_state + heating_info + neighbor_info + outside_info + wall_info
        return new_state,previous_state,heating_info,neighbor_info,outside_info,wall_info
    
    def _vector_trans(self, state_size, states_packed, actions_packed):
        new_states = []
        states = tf.unpack(states_packed)
        actions = tf.unpack(actions_packed)
        for i in range(state_size):
            new_states.append(self._transition(i,states,actions))
        return tf.pack(new_states)
    
    def Transition(self, states, actions):
        new_states = []
        batch_size,state_size = states.get_shape()
        states_list = tf.unpack(states)
        actions_list = tf.unpack(actions)
        for i in range(batch_size):
            new_states.append(self._vector_trans(state_size,states_list[i],actions_list[i]))
        return tf.pack(new_states)
                

In [8]:
adj_outside = [0,2,3,5]
adj_hall = [0,1,2,3,4,5]
adj = [[0,1],[0,3],[1,2],[1,4],[2,5],[3,4],[4,5]]
rooms = [0,1,2,3,4,5]

In [9]:
# States
states = tf.placeholder(tf.float32,[10, 6],name="States")

# Actions
actions = tf.placeholder(tf.float32,[10, 6],name="Actions")

In [10]:
hvac_inst = HVAC(adj_outside,adj_hall,adj,rooms,default_settings)

In [12]:
states_list=tf.unpack(states)
actions_list = tf.unpack(actions)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
feed_dict={states:S_A_matrix[:10,6:], actions:S_A_matrix[:10,:6]}
new_state,previous_state,heating_info,neighbor_info,outside_info,wall_info = hvac_inst._transition(1,states_list[0],states_list[0])
sess.run([new_state,previous_state,heating_info,neighbor_info,outside_info,wall_info], feed_dict=feed_dict)

[13.7725, 10.0, 3.7725003, 0.0, 0.0, 0.0]

In [23]:
hvac_inst.ADJ(4,5)

True

In [11]:
new_states=hvac_inst.Transition(states,actions)

In [12]:
states.get_shape()

TensorShape([Dimension(10), Dimension(6)])

In [13]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
feed_dict={states:S_A_matrix[:10,6:], actions:S_A_matrix[:10,:6]}
sess.run(new_states,feed_dict=feed_dict )

array([[ 12.77250004,  13.77250004,  12.77250004,  12.77250004,
         13.77250004,  12.77250004],
       [ 13.2819128 ,  15.13435936,  13.23191357,  13.23191357,
         15.1843586 ,  13.23191357],
       [ 13.7446003 ,  16.22845459,  13.60779381,  13.60779381,
         16.36526108,  13.60779381],
       [ 14.16048145,  17.10748672,  13.91539669,  13.91539669,
         17.35257149,  13.91539669],
       [ 13.08457375,  15.53688717,  13.55828094,  12.65088463,
         17.80918503,  12.97506332],
       [ 11.88453102,  17.85815048,  12.23698616,  12.82246494,
         17.01885986,  13.0997324 ],
       [ 15.56494236,  16.85007668,  13.7098093 ,  12.28947067,
         16.63779449,  13.35307121],
       [ 15.00087452,  15.95214176,  12.63172245,  12.74021339,
         16.53281784,  13.33865166],
       [ 14.04129696,  16.62973022,  12.38176727,  14.04396725,
         16.62691116,  13.88791943],
       [ 11.99556923,  18.23290634,  14.22823429,  12.31583881,
         16.82422638,  12.4

In [13]:
print(S_A_matrix[:2,:6])

[[ 10.  10.  10.  10.  10.  10.]
 [ 10.  10.  10.  10.  10.  10.]]


In [14]:
print(S_A_matrix[:2,6:])

[[ 10.      10.      10.      10.      10.      10.    ]
 [ 13.6975  13.7725  13.6975  13.6975  13.7725  13.6975]]
